In [1]:
! pip install -q kaggle

In [2]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
! cp kaggle.json ~/.kaggle/

In [7]:
! kaggle datasets download -d jangedoo/utkface-new

 97% 322M/331M [00:02<00:00, 163MB/s]
100% 331M/331M [00:02<00:00, 131MB/s]


In [8]:
! unzip -qq utkface-new.zip

In [4]:
! pip install -q wandb

In [5]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import (
    WandbMetricsLogger,
    WandbModelCheckpoint
)

In [6]:
run = wandb.init(project='Age_Prediction')

wandb: Currently logged in as: sobi-s-k2007 (sobiteam). Use `wandb login --relogin` to force relogin


In [7]:
config = wandb.config

In [8]:
wandb_callback = [
    WandbMetricsLogger(log_freq=5),
    WandbModelCheckpoint('models')
]

In [9]:
width = height = 120

In [10]:
images = []
ages = []

for image_name in os.listdir('UTKFace'):
  parts = image_name.split('_')
  age = int(parts[0])
  ages.append(age)

  image = cv2.imread('UTKFace/' + image_name)
  image = cv2.resize(image, (width, height))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  images.append(image)

In [11]:
images = pd.Series(images, name='Images')
ages = pd.Series(ages, name='Ages')

df = pd.concat((images, ages), axis=1)
df.head()

,Images,Ages
0,"[[[74, 73, 71], [59, 58, 56], [58, 57, 55], [5...",51
1,"[[[197, 158, 115], [205, 167, 124], [201, 166,...",30
2,"[[[70, 9, 0], [73, 9, 0], [77, 8, 0], [79, 6, ...",27
3,"[[[185, 164, 135], [182, 161, 134], [171, 152,...",26
4,"[[[158, 158, 132], [169, 169, 143], [168, 168,...",2


In [12]:
under_4 = df[df['Ages'] <= 4]
up_4 = df[df['Ages'] > 4]

under_4 = under_4.sample(frac=0.3)

df = pd.concat([under_4, up_4], axis=0)

In [13]:
df = df[df['Ages'] < 80]

In [14]:
X = np.array(df['Images'].values.tolist())
Y = np.array(df['Ages'].values.tolist())

In [23]:
del df

In [15]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.2)
X_train.shape, X_validation.shape, Y_train.shape, Y_validation.shape

((17214, 120, 120, 3), (4304, 120, 120, 3), (17214,), (4304,))

In [16]:
image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
)

train_data = image_data_generator.flow(
    X_train,
    Y_train,
    batch_size=32,
    shuffle=True
)

validation_data = image_data_generator.flow(
    X_validation,
    Y_validation,
    batch_size=16,
    shuffle=False
)

In [17]:
del images, ages, under_4, up_4, X, Y
del X_train, X_validation, Y_train, Y_validation

#Create Model

In [18]:
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(width, height, 3),
    pooling='avg'
)

9406464/9406464 [==============================] - 0s 0us/step


In [19]:
for layer in base_model.layers:
  layer.trainable = False

In [20]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='relu')
])

In [21]:
model.compile(loss=tf.keras.losses.mean_absolute_error,
              optimizer=tf.keras.optimizers.Adam())

In [22]:
model.fit(train_data,
          validation_data=validation_data,
          epochs=32,
          callbacks=wandb_callback)

Epoch 1/32
535/538 [============================>.] - ETA: 0s - loss: 11.9400

wandb: Adding directory to artifact (./models)... Done. 0.0s


538/538 [==============================] - 42s 64ms/step - loss: 11.9295 - val_loss: 9.9308
Epoch 2/32
536/538 [============================>.] - ETA: 0s - loss: 9.9943

wandb: Adding directory to artifact (./models)... Done. 0.0s


538/538 [==============================] - 31s 57ms/step - loss: 9.9989 - val_loss: 9.3033
Epoch 3/32
537/538 [============================>.] - ETA: 0s - loss: 9.7746

...

538/538 [==============================] - 31s 58ms/step - loss: 9.3444 - val_loss: 8.6571
Epoch 31/32
535/538 [============================>.] - ETA: 0s - loss: 9.3895

wandb: Adding directory to artifact (./models)... Done. 0.1s


538/538 [==============================] - 31s 58ms/step - loss: 9.3893 - val_loss: 8.6713
Epoch 32/32
536/538 [============================>.] - ETA: 0s - loss: 9.4088

wandb: Adding directory to artifact (./models)... Done. 0.0s


538/538 [==============================] - 31s 58ms/step - loss: 9.4044 - val_loss: 8.5682


In [23]:
model.save("/content/drive/MyDrive/PyLearn 7 Course/55.CNN_Regression/Age_Recognition_Model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
